# Random Forest

In [1]:
import numpy as np
import pandas as pd
import gc
import time
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Loads the feature matrix and sets validation strategy.

In [2]:
data = pd.read_pickle('feature_matrix.pickle')
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
del data
gc.collect();

Tune params manually.

In [3]:
ts = time.time()
model = RandomForestRegressor(
    bootstrap=True, 
    criterion='mse', 
    max_depth=15,
    n_estimators=100, 
    n_jobs=-1, 
    random_state=42, 
    warm_start=True)
model.fit(X_train, Y_train)
time.time()- ts

C:\Anaconda\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


12577.38900232315

In [4]:
# model = pickle.load(open('rf_model.pickle', 'rb'))

Dump predictions for test and validation sets to the files.

In [7]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

mean_squared_error(Y_pred, Y_valid)

0.9664852909433993

In [8]:
pickle.dump(model, open('rf_model.pickle', 'wb'))
pickle.dump(Y_pred, open('rf_train.pickle', 'wb'))
pickle.dump(Y_test, open('rf_test.pickle', 'wb'))

In [9]:
test = pd.read_csv('test.csv.gz', compression='gzip').set_index('ID')
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('rf_submission.csv', index=False)

LB: 1.08290